In [64]:
import pandas as pd
import numpy as np
import re
import os
import itertools
import torch
import ast
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset
from sklearn.model_selection import train_test_split


In [65]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)


Using device: mps


## Step (1/4): Classify original dataset to include:
1. hdb_classification (Mature/Non-Mature)
2. storey_category (Low: 1-15/High: >=16)
3. floor_area_category (in ranges of 5sqm)

In [66]:

# Define the correct file path (adjust if needed)
file_path_final = "./resale_flat_prices_original.xlsx"

# Load the Excel file
df1 = pd.read_excel(file_path_final)

# Drop missing values
df1.dropna(inplace=True)

In [67]:
# Define mature and non-mature estates
mature_estates = [
    "ANG MO KIO", "BEDOK", "BISHAN", "BUKIT MERAH", "BUKIT TIMAH", "CENTRAL AREA",
    "CLEMENTI", "GEYLANG", "KALLANG/WHAMPOA", "MARINE PARADE", "PASIR RIS",
    "QUEENSTOWN", "SERANGOON", "TAMPINES", "TOA PAYOH"
]

non_mature_estates = [
    "BUKIT BATOK", "BUKIT PANJANG", "CHOA CHU KANG", "HOUGANG", "JURONG EAST",
    "JURONG WEST", "PUNGGOL", "SEMBAWANG", "SENGKANG", "TENGAH", "WOODLANDS", "YISHUN"
]

# Function to classify towns as mature or non-mature
def classify_town(town):
    if town.upper() in mature_estates:
        return "Mature"
    elif town.upper() in non_mature_estates:
        return "Non-Mature"
    else:
        return "Unknown"  # For any town not listed

# Function to classify storey range as Low or High
def classify_storey_range(storey_range):
    # Extract the first number from the storey range (e.g., "01 TO 03" → 1)
    try:
        first_storey = int(storey_range.split(" TO ")[0])
        return "Low" if first_storey <= 15 else "High"
    except:
        return "Unknown"  # Handle unexpected format

# Function to classify floor_area_sqm into 5sqm ranges
def classify_floor_area_sqm(floor_area_sqm):
    return (floor_area_sqm // 5) * 5

# Apply classification to the dataset
df1["hdb_classification"] = df1["town"].apply(classify_town)
df1["storey_category"] = df1["storey_range"].apply(classify_storey_range)
df1["floor_area_category"] = df1["floor_area_sqm"].apply(classify_floor_area_sqm)

# Save the updated file
# output_file_path2 = "./resale_flat_prices_classified.xlsx"
# df1.to_excel(output_file_path2, index=False)

## Step (2/4): Aggregate classified dataset to group data by:
"month", "town", "flat_type", "street_name", "storey_category", "floor_area_category"

In [68]:
file_path2 = "./resale_flat_prices_classified.xlsx"
df2 = pd.read_excel(file_path2)
df2.dropna(inplace=True)

In [69]:
# Define mode function for categorical variables
def mode(series):
    return series.mode()[0] if not series.mode().empty else None

# Function to convert "XX years YY months" to total years as float
def convert_remaining_lease(lease_str):
    match = re.match(r"(\d+)\s+years\s+(\d+)\s+months", lease_str)
    if match:
        years = int(match.group(1))
        months = int(match.group(2))
        return years + (months / 12)  # Convert months to fraction of a year
    match = re.match(r"(\d+)\s+years", lease_str)
    if match:
        return int(match.group(1))  # If only years are present
    return None  # Handle unexpected formats

# Apply conversion
df2["remaining_lease"] = df2["remaining_lease"].astype(str).apply(convert_remaining_lease)

# Convert month to datetime format
df2["month"] = pd.to_datetime(df1["month"], format="%Y-%m")

# Aggregate data based on specified criteria
df2_aggregated = df2.groupby(
    ["month", "town", "flat_type", "street_name", "storey_category", "floor_area_category"]
).agg({
    "flat_model": mode,  # Most frequent flat model
    "lease_commence_date": "median",  # Median lease commence date
    "remaining_lease": "median",  # Median remaining lease
    "resale_price": "median",  # Median resale price
    "hdb_classification": mode,  # Most frequent HDB classification
}).reset_index()

# Save the updated file
# output_file_path3 = "./resale_flat_prices_aggregated.xlsx"
# df2_aggregated.to_excel(output_file_path3, index=False)

## Step (3/4): Fill up missing data by:
1. Interpolation on numerical data
2. Mode/Forward-Fill & Backward-Fill on categorical data



In [70]:
file_path3 = "./resale_flat_prices_aggregated.xlsx"
df3 = pd.read_excel(file_path3)
df3.dropna(inplace=True)

In [71]:
# Step 1: Define your grouping columns
grouping_columns = ['town', 'flat_type', 'street_name', 'storey_category', 'floor_area_category']
key_columns = ['month'] + grouping_columns

# Step 2: Extract year for filtering later
df3['year'] = df3['month'].dt.year

# Step 3: Identify actual combinations with years they existed in
group_year_combos = df3[['year'] + grouping_columns].drop_duplicates()

# Step 4: Generate full grid: only fill months from years each combo actually appeared
full_grid = pd.DataFrame([
    {'month': pd.Timestamp(f"{y}-{m:02d}-01"), 'year': y, **dict(zip(grouping_columns, combo))}
    for _, row in group_year_combos.iterrows()
    for m in range(1, 13)
    for y, *combo in [row.values]  # unpack year + combo
])

# Step 5: Identify missing rows
existing_keys = df3[key_columns].drop_duplicates()
missing_keys = pd.merge(full_grid, existing_keys, how='left', indicator=True)
missing_keys = missing_keys[missing_keys['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 6: Create missing rows with NaNs for other columns
for col in df3.columns:
    if col not in missing_keys.columns:
        missing_keys[col] = pd.NA

# Step 7: Combine and interpolate
df3_full = pd.concat([df3, missing_keys], ignore_index=True)

# Interpolate and fill
numeric_cols = ['lease_commence_date', 'remaining_lease', 'resale_price']
non_numeric_cols = ['flat_model', 'hdb_classification']
grouping_with_year = grouping_columns + ['year']

# Interpolate numeric columns ONLY where there are NaNs
for col in numeric_cols:
    df3_full[col] = df3_full.groupby(grouping_with_year)[col].transform(
        lambda x: x if x.notna().all() else x.interpolate()
    )

# Fill non-numeric columns ONLY where there are NaNs
for col in non_numeric_cols:
    df3_full[col] = df3_full.groupby(grouping_with_year)[col].transform(
        lambda x: x if x.notna().all() else x.fillna(x.mode()[0] if not x.mode().empty else pd.NA)
    )
    df3_full[col] = df3_full[col].ffill().bfill()

# Drop 'year' if still present
if 'year' in df3_full.columns:
    df3_full = df3_full.drop(columns=['year'])

df3_full = df3_full.sort_values('month').reset_index(drop=True)

# Save the interpolated dataset
output_file_path4 = "./resale_flat_prices_final.xlsx"
df3_full.to_excel(output_file_path4, index=False)

/var/folders/5x/zyxhwl456zgdlj_lrsfyy9qm0000gn/T/ipykernel_69732/1148214081.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df3_full = pd.concat([df3, missing_keys], ignore_index=True)


## Step (4/4): Encoding of non-numerical data:
1. One-hot vector encoding for town, flat_type, storey_category, flat_model and hdb_classification
2. nn.embeddings for street_name

In [72]:
file_path4 = "./resale_flat_prices_final.xlsx"
df4 = pd.read_excel(file_path4)
df4.dropna(inplace=True)

In [73]:
# Get number of unique values
num_unique_towns = df4['town'].nunique()
num_unique_flat_types = df4['flat_type'].nunique()
num_unique_storey_categories = df4['storey_category'].nunique()
num_unique_streets = df4['street_name'].nunique()
num_unique_flat_models = df4['flat_model'].nunique()
num_unique_hdb_classifications = df4['hdb_classification'].nunique()

print("Number of unique towns:", num_unique_towns)
print("Number of unique flat types:", num_unique_flat_types)
print("Number of unique storey categories:", num_unique_storey_categories)
print("Number of unique street names:", num_unique_streets)
print("Number of unique flat models:", num_unique_flat_models)
print("Number of unique HDB classifications:", num_unique_hdb_classifications)

Number of unique towns: 26
Number of unique flat types: 7
Number of unique storey categories: 2
Number of unique street names: 571
Number of unique flat models: 21
Number of unique HDB classifications: 2


In [74]:
# One-hot encode town, flat_type, storey_category, flat_model and hdb_classification
town_ohe = pd.get_dummies(df4['town']).values.astype(int).tolist()
flat_type_ohe = pd.get_dummies(df4['flat_type']).values.astype(int).tolist()
storey_category_ohe = pd.get_dummies(df4['storey_category']).values.astype(int).tolist()
flat_model_ohe = pd.get_dummies(df4['flat_model']).values.astype(int).tolist()
hdb_classification_ohe = pd.get_dummies(df4['hdb_classification']).values.astype(int).tolist()

df4['town_one_hot'] = town_ohe
df4['flat_type_one_hot'] = flat_type_ohe
df4['storey_category_one_hot'] = storey_category_ohe
df4['flat_model_one_hot'] = flat_model_ohe
df4['hdb_classification_one_hot'] = hdb_classification_ohe




# Pre-training Street Name Embeddings

Pre-training task: classify street name embedding according to town

In [75]:
# Encode street_names as indices
street_name_to_idx = {name: idx for idx, name in enumerate(df4['street_name'].unique())}
df4['street_name_idx'] = df4['street_name'].map(street_name_to_idx)

# Get towns and street names
df_town_st = df4[['town', 'street_name']].copy(deep=True)
df_town_st['street_name'] = df_town_st['street_name'].str.lower()
df_town_st['town'] = df_town_st['town'].str.lower()

# Get unique street names
unique_st_names = sorted(df_town_st['street_name'].unique().tolist())
num_st_names = len(unique_st_names)
print('Number of unique street names:', num_st_names)

# Prepare street-to-index mapping
street_to_idx = {c: i for i,c in enumerate(unique_st_names)}

# Convert streets to index labels
st_labels = [street_to_idx[st] for st in unique_st_names]
st_tensor = torch.tensor(st_labels)

# Get max street name character length
st_name_max_len = max(len(name) for name in unique_st_names)
print('Max street name length:', st_name_max_len)

# Get unique characters in street names
unique_chars = sorted(set(list(''.join(unique_st_names))))
print('Number of unique characters:', len(unique_chars))

# Prepare character-to-index mapping + reserve padding index
char_to_idx = {c: i for i,c in enumerate(unique_chars, start=1)}
char_to_idx['<pad>'] = 0
vocab_size = len(char_to_idx)
print('Vocab size:', vocab_size)

# Convert street names to padded arrays of indices
char_arr = [[char_to_idx[c] for c in name] for name in unique_st_names]
char_arr = [(arr + [0] * (st_name_max_len - len(arr))) for arr in char_arr]
char_tensor = torch.tensor(char_arr)
print('Character tensor shape:', char_tensor.shape)

# Get town as street name labels
town_labels = []
for name in unique_st_names:
    df = df_town_st[df_town_st['street_name'] == name].reset_index()
    town_labels.append(df.loc[0, 'town'])

# Get number of unique towns
unique_towns = sorted(set(town_labels))
num_towns = len(unique_towns)
print('Number of towns:', num_towns)

# Prepare town-to-index mapping
town_to_idx = {c: i for i,c in enumerate(unique_towns)}

# Convert town labels to indices
town_labels = [town_to_idx[town] for town in town_labels]
town_label_tensor = torch.tensor(town_labels)
print('Label tensor shape:', town_label_tensor.shape)

Number of unique street names: 571
Max street name length: 20
Number of unique characters: 38
Vocab size: 39
Character tensor shape: torch.Size([571, 20])
Number of towns: 26
Label tensor shape: torch.Size([571])


In [76]:
# split street name dataset to train and test sets
train_feats, test_feats, train_labels, test_labels = train_test_split(char_tensor, town_label_tensor, test_size=0.2, shuffle=True)

# Convert to tensor datasets
train_dset = TensorDataset(train_feats, train_labels)
test_dset = TensorDataset(test_feats, test_labels)

# Load datasets
bs = 16
train_loader = DataLoader(train_dset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=bs, shuffle=False)

In [77]:
class CharRNNencoder(nn.Module):
    """
    GRU-based character-based embedding layer for street name encoding
    """
    def __init__(self, vocab_size, char_dim, hidden_dim, num_layers=1, bidirectional=True):
        super().__init__()
        self.char_emb = nn.Embedding(vocab_size, char_dim, padding_idx=0)
        self.rnn = nn.GRU(input_size=char_dim,
                           hidden_size=hidden_dim,
                           num_layers=num_layers,
                           bidirectional=bidirectional,
                           batch_first=True)
        
    def forward(self, x):
        # x: [batch, name_len]
        emb = self.char_emb(x)               # [batch, name_len, char_dim]
        outputs, hidden = self.rnn(emb)    # hidden: [num_layers*dirs, batch, hidden_dim]
        # Concatenate final forward/backward hidden states
        if self.rnn.bidirectional:
            hidden = hidden.view(self.rnn.num_layers, 2, x.size(0), -1)
            hidden = torch.cat([hidden[-1,0], hidden[-1,1]], dim=1)  # [batch, 2*hidden_dim]
        else:
            hidden = hidden[-1]  # [batch, hidden_dim]
        return hidden                            


### Pre-training loop

In [78]:
def train(train_loader, embed_model, embed_optimizer, clf_model, clf_optimizer, criterion, device):
    "Per-epoch embedding-classifier model pre-training procedure"

    # Set models to trainig mode
    embed_model.train()
    clf_model.train()

    train_loss = 0
    for batch in train_loader:
        # Extract inputs and labels
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Reset gradieints
        embed_optimizer.zero_grad()
        clf_optimizer.zero_grad()

        # Forward pass
        embeddings = embed_model(inputs)
        logits = clf_model(embeddings)

        # Compute loss
        loss = criterion(logits, labels)

        # Back propagation
        loss.backward()

        # Update parameters
        clf_optimizer.step()
        embed_optimizer.step()

        # Compile losses
        train_loss += loss.item()
    
    # Calculate average loss
    avg_train_loss = train_loss / len(train_loader)

    return avg_train_loss
    
def evaluate(val_loader, embed_model, clf_model, loss_fn, device):
    "Per-epoch embedding-classifier model evaluation procedure"
    # Set to eval mode
    embed_model.eval()
    clf_model.eval()

    correct, total = 0, 0
    val_loss = 0
    # Freeze gradient during calculation of losses and accuracies
    with torch.no_grad():
        for batch in val_loader:
            # Extract inputs and labels
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            embeddings = embed_model(inputs)
            logits = clf_model(embeddings)

            # Inferencing / predictions
            val_loss += loss_fn(logits, labels).item()
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    # Calculate performance metrics
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total

    return avg_val_loss, val_accuracy

In [79]:
bidirectional = True
char_dim = 8        # hyperparameter
embed_size = 16     # fixed

# Embedding layer
embed_model = CharRNNencoder(
    vocab_size=len(char_to_idx),
    char_dim=char_dim,
    hidden_dim=(embed_size // 2 if bidirectional else embed_size),
    num_layers=1,
    bidirectional=True
).to(device)

clf_hidden_size = 32    # hyperparameter

# Classifier
classifier = nn.Sequential(
    nn.Linear(embed_size, clf_hidden_size),
    nn.ReLU(),
    nn.Dropout1d(p=0.5),
    nn.Linear(clf_hidden_size, num_towns)
).to(device)

# Training hyperparameters
epochs = 500
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
embed_optimizer = optim.Adam(embed_model.parameters(), lr=0.001)
clf_optimizer = optim.Adam(classifier.parameters(), lr=0.001)  

In [80]:
# Keep track of losses and accuracies
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
    
for epoch in range(epochs):
    # Train models
    train(
        train_loader,
        embed_model,
        embed_optimizer,
        classifier,
        clf_optimizer,
        criterion,
        device
    )

    # Evaluate accuracies and losses
    avg_train_loss, train_accuracy = evaluate(
        train_loader,
        embed_model,
        classifier,
        nn.functional.cross_entropy,
        device
    )

    avg_val_loss, val_accuracy = evaluate(
        test_loader,
        embed_model,
        classifier,
        nn.functional.cross_entropy,
        device
    )

    # Save accuracies and losses
    train_loss_history.append(avg_train_loss)
    train_acc_history.append(train_accuracy)

    train_loss_history.append(avg_train_loss)
    train_acc_history.append(train_accuracy)
    val_loss_history.append(avg_val_loss)
    val_acc_history.append(val_accuracy)

    if (epoch+1) % 25 == 0:
        # Display training and validation loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}: Avg. train loss = {avg_train_loss:.4f} | Avg. train acc. = {train_accuracy:.4f} | Avg. val. loss = {avg_val_loss:.4f} | Avg. val. acc. = {val_accuracy:.4f}')

        

# Set models to eval mode for evaluation 
embed_model.eval()
classifier.eval()

Epoch 25/500: Avg. train loss = 1.9072 | Avg. train acc. = 0.6031 | Avg. val. loss = 2.2816 | Avg. val. acc. = 0.5130
Epoch 50/500: Avg. train loss = 1.5010 | Avg. train acc. = 0.6864 | Avg. val. loss = 1.9728 | Avg. val. acc. = 0.5478
Epoch 75/500: Avg. train loss = 1.2846 | Avg. train acc. = 0.7325 | Avg. val. loss = 1.8759 | Avg. val. acc. = 0.5565
Epoch 100/500: Avg. train loss = 1.0907 | Avg. train acc. = 0.7873 | Avg. val. loss = 1.8203 | Avg. val. acc. = 0.6000
Epoch 125/500: Avg. train loss = 0.9472 | Avg. train acc. = 0.8443 | Avg. val. loss = 1.7597 | Avg. val. acc. = 0.6261
Epoch 150/500: Avg. train loss = 0.8292 | Avg. train acc. = 0.8794 | Avg. val. loss = 1.7667 | Avg. val. acc. = 0.6435
Epoch 175/500: Avg. train loss = 0.7139 | Avg. train acc. = 0.9101 | Avg. val. loss = 1.8162 | Avg. val. acc. = 0.6609
Epoch 200/500: Avg. train loss = 0.6204 | Avg. train acc. = 0.9496 | Avg. val. loss = 1.8259 | Avg. val. acc. = 0.6783
Epoch 225/500: Avg. train loss = 0.5612 | Avg. trai

Sequential(
  (0): Linear(in_features=16, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout1d(p=0.5, inplace=False)
  (3): Linear(in_features=32, out_features=26, bias=True)
)

In [81]:
# Generate street name embeddings
st_name_embeddings = embed_model(char_tensor.to(device)).detach().cpu().numpy().tolist()
print(st_name_embeddings)

[[-0.9256269335746765, -0.9982683062553406, 0.4340050220489502, 0.6986680626869202, -0.9910224676132202, 0.4261637330055237, -0.9509984850883484, 0.9944260716438293, -0.6615110039710999, 0.33562541007995605, 0.8225889205932617, 0.16580170392990112, -0.5675117373466492, -0.5285341143608093, -0.10733973979949951, -0.9740935564041138], [-0.8508911728858948, -0.9962965846061707, 0.5517258048057556, 0.8700464367866516, -0.8393427729606628, 0.7660667300224304, -0.8480182886123657, 0.9846225380897522, -0.6728663444519043, 0.3766314387321472, 0.8170906901359558, 0.18790915608406067, -0.5696348547935486, -0.5374492406845093, -0.0930633544921875, -0.975833535194397], [-0.9962167143821716, -0.9977923631668091, -0.868023693561554, -0.9884024262428284, -0.9936035871505737, -0.7518430948257446, -0.9999361634254456, 0.9995003342628479, 0.9120182394981384, -0.8005422353744507, -0.6975744962692261, 0.7022744417190552, 0.994581937789917, 0.9945237636566162, 0.1959685981273651, 0.8494002223014832], [-0.8

In [82]:
# Map street name to corresponding embeddings
df4['street_name_embed'] = df4['street_name_idx'].apply(lambda i: st_name_embeddings[i])

In [83]:
# Save the final DataFrame with one-hot vectors and embeddings
output_file_path5 = "./resale_flat_prices_one_hot_vector_and_embed.xlsx"
df4.to_excel(output_file_path5, index=False)